# Data Clean & Pre-process

Import Libraries

In [2]:
import os
import pandas as pd
import numpy as np
from opencage.geocoder import OpenCageGeocode
import folium

Update Directories

In [3]:
#Update the Database with relative paths
current_dir = os.getcwd()
task03_dir = os.path.dirname(current_dir)
database_dir = os.path.join(task03_dir, 'Database')
cleaned_data_dir = os.path.join(database_dir, 'Cleaned Data')

Get Data from CSV

In [4]:
#Colombo Motor Show Attendance
colombo_motor_show_attendance = pd.read_csv(os.path.join(cleaned_data_dir, 'ColomboMotorShowAttendees.csv'))

#Weather Climate DAta
weather_climate_data = pd.read_csv(os.path.join(cleaned_data_dir, 'WeatherClimateData.csv'))

In [5]:
colombo_motor_show_attendance

,eventName,eventLocation,attndName,attndLocation,attndDate,attndRefHashTag
0,Colombo Motor Show,"BMICH, Colombo",Harshana Kalinga,"Kaduwela, Sri Lanka",11/18/2023,#ColomboMotorShow
1,Colombo Motor Show,"BMICH, Colombo",Sadish Pathirana,"Narammala, Sri Lanka",11/17/2023,#ColomboMotorShow
2,Colombo Motor Show,"BMICH, Colombo",Malith Pramodya Palliyaguruge,"Hakmana, Sri Lanka",11/18/2023,#ColomboMotorShow
3,Colombo Motor Show,"BMICH, Colombo",Shiromi Rathnayake,"Colombo, Sri Lanka",11/17/2023,#ColomboMotorShow
4,Colombo Motor Show,"BMICH, Colombo",Dilshan Subasinghe,"Galle, Sri Lanka",11/18/2023,#ColomboMotorShow
...,...,...,...,...,...,...
94,Colombo Motor Show,"BMICH, Colombo",Kavindu Katuwandeniya,"Mathara, Sri Lanka",11/18/2023,#ColomboMotorShow
95,Colombo Motor Show,"BMICH, Colombo",Nethmi Jayasekara,"Mathara, Sri Lanka",11/18/2023,#ColomboMotorShow
96,Colombo Motor Show,"BMICH, Colombo",Shehan Perera,"Kaduwela, Sri Lanka",11/19/2023,#ColomboMotorShow
97,Colombo Motor Show,"BMICH, Colombo",Jeewantha Ariyasinghe,"Kegalle, Sri Lanka",11/19/2023,#ColomboMotorShow


In [6]:
weather_climate_data

,eventLocation,attndDate,dhTemp (C),dlTemp (C),precipitation (mm),humidity,wind (km/h)
0,"BMICH, Colombo",11/17/2023,32,26,4.3,0.8684,5.3
1,"BMICH, Colombo",11/18/2023,31,25,6.7,0.8834,6.3
2,"BMICH, Colombo",11/19/2023,31,24,4.1,0.8937,6.9
3,"BMICH, Colombo",11/20/2023,31,24,2.7,0.8805,5.0
4,"BMICH, Colombo",11/21/2023,32,25,5.6,0.8893,6.3


In [7]:
#Set the date format explicitly
date_format = '%m/%d/%Y'

#Convert to datetime
colombo_motor_show_attendance['attndDate'] = pd.to_datetime(colombo_motor_show_attendance['attndDate'], format=date_format)
weather_climate_data['attndDate'] = pd.to_datetime(weather_climate_data['attndDate'], format=date_format)

## Get Location Data

In [8]:
#Opencage API Key
api_key = '345da8ea035c4f6490f6bd652d926602'

In [10]:
unique_event_locations = colombo_motor_show_attendance['eventLocation'].unique()
unique_attnd_locations = colombo_motor_show_attendance['attndLocation'].unique()

#Get the unique locations
unique_locations = pd.unique(np.concatenate((unique_event_locations, unique_attnd_locations), axis=0))

#Get the coordinates for the unique locations
geocoder = OpenCageGeocode(api_key)

#Get the coordinates for the unique location
coordinates = []
for location in unique_locations:
    results = geocoder.geocode(location)
    if results and len(results):
        coordinates.append(results[0]['geometry'])
    else:
        coordinates.append(None)

#Create a dataframe with the coordinates
coordinates_df = pd.DataFrame(coordinates)
coordinates_df

#Add the coordinates to the unique locations
unique_locations_df = pd.DataFrame(unique_locations, columns=['location'])
unique_locations_df = pd.concat([unique_locations_df, coordinates_df], axis=1)
unique_locations_df

,location,lat,lng
0,"BMICH, Colombo",6.901107,79.873618
1,"Kaduwela, Sri Lanka",6.935703,79.984331
2,"Narammala, Sri Lanka",7.430730,80.214379
3,"Hakmana, Sri Lanka",6.083926,80.644974
4,"Colombo, Sri Lanka",6.938861,79.854201
5,"Galle, Sri Lanka",6.032814,80.214955
6,"Kurunegala, Sri Lanka",7.487046,80.364908
7,"Wattala, Sri Lanka",6.989871,79.892709
8,"Homagama, Sri Lanka",6.841238,80.003446
9,"Malabe, Sri Lanka",6.904072,79.954619


In [11]:
#Save the unique locations Data as a CSV file
unique_locations_df.to_csv(os.path.join(cleaned_data_dir, 'Locations.csv'), index=False)